In [1]:
import pandas as pd
import duckdb as duck
import liquidation_schedule as ls

In [2]:
assets = {
    'Asset Name': ['Asset1', 'Asset2', 'Asset3', 'Asset4'],
    'Currency': ['USD', 'USD', 'SGD', 'SGD'],
    'Notional': [1_000_000, 2_000_000, 3_000_000, 4_000_000]
}

holiday = {
    'Currency': ['SGD', 'SGD', 'SGD', 'SGD', 'USD', 'USD', 'USD', 'USD'],
    'Date': ['2024-08-03', '2024-08-04', '2024-08-10', '2024-08-11', '2024-08-03', '2024-08-04', '2024-08-10', '2024-08-11']
}

liquidation_schedule = {
    'Asset Name': ['Asset1', 'Asset2', 'Asset3', 'Asset4'],
    'Liquidating Start Date': ['2024-08-06', '2024-08-08', '2024-08-08', '2024-08-05'],
    'No of days liquidation': [3, 2, 6, 5],
    'Expected Cash Flow Start Date': ['2024-08-07', '2024-08-09', '2024-08-12', '2024-08-06']
}

assets_table = pd.DataFrame(assets)
holiday_table = pd.DataFrame(holiday)
holiday_table['Date'] = pd.to_datetime(holiday_table['Date'])
liquidation_schedule_table = pd.DataFrame(liquidation_schedule)
liquidation_schedule_table['Liquidating Start Date'] = pd.to_datetime(liquidation_schedule_table['Liquidating Start Date'])
liquidation_schedule_table['Expected Cash Flow Start Date'] = pd.to_datetime(liquidation_schedule_table['Expected Cash Flow Start Date'])

In [3]:
display(assets_table); display(holiday_table); display(liquidation_schedule_table)

,Asset Name,Currency,Notional
0,Asset1,USD,1000000
1,Asset2,USD,2000000
2,Asset3,SGD,3000000
3,Asset4,SGD,4000000


,Currency,Date
0,SGD,2024-08-03
1,SGD,2024-08-04
2,SGD,2024-08-10
3,SGD,2024-08-11
4,USD,2024-08-03
5,USD,2024-08-04
6,USD,2024-08-10
7,USD,2024-08-11


,Asset Name,Liquidating Start Date,No of days liquidation,Expected Cash Flow Start Date
0,Asset1,2024-08-06,3,2024-08-07
1,Asset2,2024-08-08,2,2024-08-09
2,Asset3,2024-08-08,6,2024-08-12
3,Asset4,2024-08-05,5,2024-08-06


# Python Version

In [4]:
liquidation_profile = ls.liquidation(assets = assets_table, holiday = holiday_table, liquidation_schedule = liquidation_schedule_table)
asset_count = 4
schedule_df = pd.DataFrame()
for i in range(asset_count):
    schedule_df = pd.concat([schedule_df, liquidation_profile.run(f'Asset{i+1}')])

schedule_df.reset_index(drop = True, inplace = True)
schedule_df

,Asset Name,Currency,Expected cash flow Amount,Expected Cash Flow Date
0,Asset1,USD,333333.333333,2024-08-07
1,Asset1,USD,333333.333333,2024-08-08
2,Asset1,USD,333333.333333,2024-08-09
3,Asset2,USD,1000000.000000,2024-08-09
4,Asset2,USD,1000000.000000,2024-08-12
5,Asset3,SGD,500000.000000,2024-08-12
6,Asset3,SGD,500000.000000,2024-08-13
7,Asset3,SGD,500000.000000,2024-08-14
8,Asset3,SGD,500000.000000,2024-08-15
9,Asset3,SGD,500000.000000,2024-08-16


# SQL Version

In [5]:
duck.query(
'''
CREATE OR REPLACE TEMP MACRO liquidate(asset) AS TABLE (

    WITH RECURSIVE 
    assetdetails AS (
        SELECT *
        FROM 
            assets_table t1
        LEFT JOIN 
            liquidation_schedule_table t2
        ON 
            t1."Asset Name" = t2."Asset Name"
        WHERE t1."Asset Name" = asset
    ),

    cash_flows AS (
        SELECT 
            "Asset Name", 
            Notional / "No of days liquidation" AS "Expected cash flow Amount" 
        FROM 
            assetdetails
    ),

    holidays AS (
        SELECT * 
        FROM
            holiday_table
        WHERE
            Currency IN (
                SELECT 
                    Currency
                FROM
                    assets_table 
                WHERE
                    "Asset Name" = asset
            )
    ), 

    DateSeq AS (
        SELECT 
            "Expected Cash Flow Start Date" as date,
            "No of days liquidation" AS 'day_num'
        FROM
            liquidation_schedule_table
        WHERE
            "Asset Name" = asset

        UNION ALL

        SELECT
            date + INTERVAL 1 DAY,
            CASE 
                WHEN (date + INTERVAL 1 DAY) NOT IN (SELECT Date FROM holidays) THEN day_num - 1
            ELSE day_num
            END
        FROM
            DateSeq
        WHERE
            ---(date + INTERVAL 1 DAY) NOT IN (SELECT Date FROM holidays) AND
            day_num > 1
    ),
    
    liquidation_dates AS (
        SELECT
            MIN(date) as "Expected Cash Flow Date" FROM DateSeq
        GROUP BY day_num
        ORDER BY "Expected Cash Flow Date"
    
    )

    SELECT 
        cf."Asset Name", 
        ad.Currency, 
        cf."Expected cash flow Amount", 
        ld."Expected Cash Flow Date"
    FROM
        liquidation_dates ld, cash_flows cf, assetdetails ad
);

SELECT * FROM liquidate('Asset1')
UNION ALL
SELECT * FROM liquidate('Asset2')
UNION ALL
SELECT * FROM liquidate('Asset3')  
UNION ALL
SELECT * FROM liquidate('Asset4')  

'''
)

┌────────────┬──────────┬───────────────────────────┬─────────────────────────┐
│ Asset Name │ Currency │ Expected cash flow Amount │ Expected Cash Flow Date │
│  varchar   │ varchar  │          double           │      timestamp_ns       │
├────────────┼──────────┼───────────────────────────┼─────────────────────────┤
│ Asset1     │ USD      │         333333.3333333333 │ 2024-08-07 00:00:00     │
│ Asset1     │ USD      │         333333.3333333333 │ 2024-08-08 00:00:00     │
│ Asset1     │ USD      │         333333.3333333333 │ 2024-08-09 00:00:00     │
│ Asset2     │ USD      │                 1000000.0 │ 2024-08-09 00:00:00     │
│ Asset2     │ USD      │                 1000000.0 │ 2024-08-12 00:00:00     │
│ Asset3     │ SGD      │                  500000.0 │ 2024-08-12 00:00:00     │
│ Asset3     │ SGD      │                  500000.0 │ 2024-08-13 00:00:00     │
│ Asset3     │ SGD      │                  500000.0 │ 2024-08-14 00:00:00     │
│ Asset3     │ SGD      │               